In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from utils import *

In [ ]:
countries = pd.read_csv("../data/2. Atlas/countries.csv", encoding="latin1")

In [ ]:
trustworthiness = pd.read_csv("../data/trustworthiness_scores.csv", dtype={"cmd": str})

discrepancy = []
for i, y in enumerate(range(2017, 2023), start=1):
    _ = pd.read_csv("../data/4. Additional Data Sources/4.2.{i} DISCREPANCY_INDEX_H5_{y}_csv.zip", dtype={"ProductCode": str})
    _ = _[["CountryA", "ProductCode", "Year", "DI"]]
    discrepancy.append(_)
discrepancy = pd.concat(discrepancy, ignore_index=True)

In [ ]:
trustworthiness.head()

In [ ]:
discrepancy.head()

Convert all discrepancies to positive (ignore the 'direction' of the discrepancy)

In [ ]:
discrepancy.loc[:, "DI"] = np.abs(discrepancy.DI)

Reduce ProductCode to single 2 digits

In [ ]:
discrepancy.loc[:, "ProductCode"] = discrepancy["ProductCode"].str[:2]

Aggregate by Year & Commodity

In [ ]:
discrepancy = discrepancy.groupby(["CountryA", "Year", "ProductCode"]).mean().reset_index()

Add Country ID

In [ ]:
discrepancy = discrepancy.merge(countries[["iso_code", "country_id"]], left_on="CountryA", right_on="iso_code", how="inner")
discrepancy.head()

In [ ]:
discrepancy.rename(columns={"Year": "year", "ProductCode": "cmd"}, inplace=True)

In [ ]:
trust_discr = trustworthiness.merge(discrepancy[["country_id", "year", "cmd", "DI"]], on=["country_id", "year", "cmd"], how="inner")
trust_discr.head()

In [ ]:
trust_discr.DI.describe()

In [ ]:
spearmanr(trust_discr["DI"], trust_discr["trustworthiness"])

### Corruption Index

In [ ]:
corruption = pd.read_csv("../data/4. Additional Data Sources/4.3 Corruption Perception Index.csv")

In [ ]:
corruption.head()

In [ ]:
corruption = corruption.rename(columns={"ISO3": "iso_code"})

In [ ]:
corruption = corruption.merge(countries[["iso_code", "country_id"]], on="iso_code", how="inner")

In [ ]:
corruption = corruption.melt(id_vars=["country_id", "iso_code"], var_name="year", value_vars=[f"{x}" for x in range(2012, 2024)])
corruption["year"] = corruption["year"].astype(int)

In [ ]:
corruption.head()

In [ ]:
trust_discr_avg = trust_discr.groupby(["country_id", "year"])[["trustworthiness", "DI"]].mean().reset_index()
trust_discr_avg.head()

In [ ]:
trust_discr_corrup = corruption.merge(trust_discr_avg, on=["country_id", "year"], how="inner")
trust_discr_corrup.head()

In [ ]:
trust_discr_corrup.dropna(inplace=True)

In [ ]:
print(spearmanr(trust_discr_corrup["trustworthiness"], trust_discr_corrup["value"]))
print(spearmanr(trust_discr_corrup["DI"], trust_discr_corrup["value"]))